In [3]:
obc_file = "/home/benr/wqmodels/ssm/wqm-test/inputs/ssm_obc_wq.dat"
out_file = "data/ssm_obc_2014.nc"

import numpy as np
from netCDF4 import Dataset

In [12]:
with open(obc_file) as f:
    num_bns = np.loadtxt([next(f)], dtype=int, comments='!')
    # The first column is just a counter, ignore
    nodes = np.loadtxt([next(f) for n in range(num_bns)], dtype=int)[:,1]
    
    # The water quality variables are the same as ssm_read_pnt except discharge
    statevars = ('temp', 'salt', 'tss',  'alg1', 'alg2', 'alg3', 'zoo1',
                 'zoo2', 'ldoc', 'rdoc', 'lpoc', 'rpoc', 'nh4',  'no32',
                 'urea', 'ldon', 'rdon', 'lpon', 'rpon', 'po4',  'ldop',
                 'rdop', 'lpop', 'rpop', 'pip',  'cod',  'doxg', 'psi',
                 'dsi',  'alg1p','alg2p','alg3p','dic',  'talk')
    statedata = {}
    for v in statevars:
        statedata[v] = []
    
    times = []
    t_line = next(f)
    while True:
        times.append(float(t_line))
        for v in statevars:
            # Each line contains a state variable counter (which we don't need because
            # it's implied) and the concentrations at one boundary node's sigma layers
            statedata[v].append(np.loadtxt([next(f) for l in range(num_bns)])[:,1:])
        # Look for the next time index, which might fail if we're at the end-of-file
        try:
            t_line = next(f)
        except StopIteration:
            break

times = np.array(times)
for v in statevars:
    statedata[v] = np.array(statedata[v])

print("Times:", times.shape)
print("Nodes:", nodes.shape)
print("NO32:", statedata['no32'].shape)
statedata['no32']

Times: (14,)
Nodes: (87,)
NO32: (14, 87, 10)


array([[[0.1727, 0.1736, 0.1761, ..., 0.1823, 0.1823, 0.1823],
        [0.2069, 0.2094, 0.214 , ..., 0.2236, 0.2236, 0.2236],
        [0.2335, 0.2342, 0.2349, ..., 0.2392, 0.2392, 0.2392],
        ...,
        [0.0756, 0.0756, 0.0753, ..., 0.0884, 0.172 , 0.172 ],
        [0.0687, 0.0689, 0.0689, ..., 0.0758, 0.0765, 0.0765],
        [0.0572, 0.0574, 0.0574, ..., 0.0584, 0.0584, 0.0584]],

       [[0.1727, 0.1736, 0.1761, ..., 0.1823, 0.1823, 0.1823],
        [0.2069, 0.2094, 0.214 , ..., 0.2236, 0.2236, 0.2236],
        [0.2335, 0.2342, 0.2349, ..., 0.2392, 0.2392, 0.2392],
        ...,
        [0.0756, 0.0756, 0.0753, ..., 0.0884, 0.172 , 0.172 ],
        [0.0687, 0.0689, 0.0689, ..., 0.0758, 0.0765, 0.0765],
        [0.0572, 0.0574, 0.0574, ..., 0.0584, 0.0584, 0.0584]],

       [[0.1513, 0.1509, 0.15  , ..., 0.1458, 0.1458, 0.1458],
        [0.177 , 0.1761, 0.1741, ..., 0.1672, 0.1672, 0.1672],
        [0.1993, 0.1984, 0.1968, ..., 0.1906, 0.1906, 0.1906],
        ...,
        [0.0

In [13]:
cdf = Dataset(out_file, "w")

time_dim = cdf.createDimension("time", len(times))
node_dim = cdf.createDimension("node", num_bns)
siglay_dim = cdf.createDimension("siglay", statedata['temp'].shape[2])

time_var = cdf.createVariable("time", "f4", ("time",))
time_var.unit = "days"
cdf['time'][:] = times
node_var = cdf.createVariable("node", "i4", ("node",))
cdf['node'][:] = nodes

for v in statevars:
    var = cdf.createVariable(v, "f4", ("time","node","siglay"))
    cdf[v][:] = statedata[v]

cdf

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): time(14), node(87), siglay(10)
    variables(dimensions): float32 time(time), int32 node(node), float32 temp(time, node, siglay), float32 salt(time, node, siglay), float32 tss(time, node, siglay), float32 alg1(time, node, siglay), float32 alg2(time, node, siglay), float32 alg3(time, node, siglay), float32 zoo1(time, node, siglay), float32 zoo2(time, node, siglay), float32 ldoc(time, node, siglay), float32 rdoc(time, node, siglay), float32 lpoc(time, node, siglay), float32 rpoc(time, node, siglay), float32 nh4(time, node, siglay), float32 no32(time, node, siglay), float32 urea(time, node, siglay), float32 ldon(time, node, siglay), float32 rdon(time, node, siglay), float32 lpon(time, node, siglay), float32 rpon(time, node, siglay), float32 po4(time, node, siglay), float32 ldop(time, node, siglay), float32 rdop(time, node, siglay), float32 lpop(time, node, siglay), float32 rpop(tim

In [14]:
cdf.close()